In [ ]:
%matplotlib widget
# Documentation needed for ipyml - notifications on push from github

from bmcs_utils.api import InteractiveModel, InteractiveWindow, Item, View
from bmcs_beam.models.moment_curvature.moment_curvature_ import MomentCurvature

import numpy as np
import sympy as sp
import traits.api as tr
import matplotlib.pylab as plt

from matplotlib.path import Path
import matplotlib.patches as mpatches
from matplotlib.patches import PathPatch
import ipywidgets as widgets

from scipy.integrate import cumtrapz
sp.init_printing()

from scipy.signal import find_peaks
from sympy.physics.continuum_mechanics.beam import Beam
# https://docs.sympy.org/latest/modules/physics/continuum_mechanics/beam_problems.html


The external stresses at a point in the finite element and along the crack are shown in  figure and the far field stress profile at a point in a finite element is represented in equation.
\begin{align}
{\sigma}_{ij} = 
\left[
\begin{array}{cc}
\sigma_{x} & \tau^\mathrm{tip} \\
\tau^\mathrm{tip} & \sigma_{z}
\end{array}
\right]
\label{eq:sigmaxz}
\end{align}
Here, $\sigma_x$ represents the far field stress that can be assumed to be equivalent to the normalized applied load. Therefore, for a three point bend beam it is taken as:
\begin{equation}
\sigma_x = \frac{P s}{b^2}
\end{equation}
where $P, s, b$ represents the load, span and breadth of the beam respectively. Furthermore, $\sigma_z$ is considered to be zero in this simulation whereas $\tau^\mathrm{tip}$ can be obtained from the shear profile of the section. Hence:
\begin{align}
{\sigma}_{ij} = 
\left[
\begin{array}{cc}
\frac{P s}{b^2} & \tau^\mathrm{tip} \\
\tau^\mathrm{tip} & 0
\end{array}
\right]
\label{eq:sigmaij}
\end{align}

<img src="fig/crack_model.png" width="800"/>

Figure shows cohesive crack model with mode I and boundary conditions acting along center line i.e. zero stress on the notch, softening equation on the cohesive zone whereas zero opening along the uncracked ligament and zero stress intensity factor at the cohesive zone front.

In [ ]:
class FarFieldStress(InteractiveModel):
    
    name = 'FarFieldStress'
   
    P = tr.Int(5000)
    s = tr.Int(600)
    b = tr.Int(200)
    tau_tip = tr.Int(10)
    #ipw_view = View(
    #    Item('P', latex='P \mathrm{[N]}', minmax=(1000,10000)),
    #    Item('s', latex='s \mathrm{[mm]}', minmax=(100,1000)),
    #    Item('b', latex='b \mathrm{[mm]}', minmax=(100,500))
    #)
    def __init__(self,P,s,b):
        self.P = P
        self.s = s
        self.b = b
        
    def get_sigma_ij(self):
        sigma_x = (self.P * self.s) / (self.b**2)
        tau_tip = self.tau_tip
        sigma_ij = np.matrix([[sigma_x, tau_tip], [tau_tip, 0]])
        return sigma_ij
    

In [ ]:
f = FarFieldStress(1,10,1)
print(f.get_sigma_ij())

In [ ]:
f = FarFieldStress(1,10,1)
class SIF(InteractiveModel):
    
    name = 'SIF'
    
    beta = tr.Int(0)
    a = tr.Int(1)
    def get_sif_K_I(self):
        R_beta_kl = np.matrix([[np.cos(self.beta), - np.sin(self.beta)], [np.sin(self.beta), np.cos(self.beta)]])
        sigma_beta_kl = R_beta_kl.T * f.get_sigma_ij() * R_beta_kl 
        K_I = np.sqrt(np.pi * self.a) * sigma_beta_kl[0,0]
        K_II = np.sqrt(np.pi * self.a) * sigma_beta_kl[1,0]
        return K_I
    
    def get_sif_K_II(self):
        R_beta_kl = np.matrix([[np.cos(self.beta), - np.sin(self.beta)], [np.sin(self.beta), np.cos(self.beta)]])
        sigma_beta_kl = R_beta_kl.T * f.get_sigma_ij() * R_beta_kl 
        K_II = np.sqrt(np.pi * self.a) * sigma_beta_kl[1,0]
        return K_II

In [ ]:
s = SIF()
print(s.get_sif_K_I())

In [ ]:
f = FarFieldStress(1,10,1)
s = SIF()
class CrackOrientationAngle:
    
    name = 'CrackOrientationAngle'
    
    def get_crack_angle(self):
        
        theta = sp.Symbol(r'\theta', real = True)
        c_t = sp.Symbol('c_t')
        s_t = sp.Symbol('c_t')
        s_3t = 3 * s_t - 4 * s_t**3
        c_3t = 4 * c_t**3 - 3 * c_t
    
        S_I_11 = 2 * c_t - c_t * (1 + s_t * s_3t)
        S_I_22 = c_t * (1 + s_t * s_3t)
        S_I_12 = c_t * s_t * c_3t
        S_I = sp.Matrix([[S_I_11, S_I_12],[S_I_12, S_I_22]])
        S_I_ = sp.simplify(S_I)

        S_II_11 = - 2 * s_t - s_t * c_t * c_3t
        S_II_22 = s_t * c_t * c_3t
        S_II_12 = c_t * (1 - s_t * s_3t)
        S_II = sp.Matrix([[S_II_11, S_II_12],[S_II_12, S_II_22]])
        S_II_ = sp.simplify(S_II)

        c_t_h = sp.sqrt((1 + c_t)/2)
        s_t_h = sp.sqrt((1 - c_t)/2)

        sigma_tip_ij = sp.Matrix([[s.get_sif_K_I() * S_I_[0,0] + s.get_sif_K_II() * S_II_[0,0], \
                                   s.get_sif_K_I() * S_I_[0,1] + s.get_sif_K_II() * S_II_[0,1]],
                   [s.get_sif_K_I() * S_I_[0,1] + s.get_sif_K_II() * S_II_[0,1], \
                    s.get_sif_K_I() * S_I_[1,1] + s.get_sif_K_II() * S_II_[1,1]]])
        cs_theta = sp.Matrix([c_t, s_t])
        R_theta_kl = sp.Matrix([[cs_theta[0], -cs_theta[1]],[cs_theta[1], cs_theta[0]]])

        sig_tip_theta_ij = R_theta_kl.T * sigma_tip_ij * R_theta_kl
        #sig_tip_theta_ij_ = sp.simplify(sig_tip_theta_ij)
        s_t_ = sp.sqrt(1 - c_t**2)
        sig_tip_theta_ij_sub = sig_tip_theta_ij.subs(s_t, s_t_)
        #sig_tip_theta_ij_sim = sp.simplify(sig_tip_theta_ij_sub)
        sig_tip_kl = sig_tip_theta_ij_sub.subs(sp.cos(theta), c_t)
        cos_theta_ = sp.solve(sig_tip_kl[1,0], c_t)
        print(sig_tip_theta_ij_sub)
        theta_ = sp.acos(cos_theta_)
        return theta_
    

In [ ]:
co = CrackOrientationAngle()
print(co.get_crack_angle())

In [ ]:
print(c.get_crack_angle())

In [ ]:
windows = InteractiveWindow([FarFieldStress()])
windows.interact()